# Translacja
Chodzi o to żeby przekształcić tekst, aby dało się go wyrenderować przy pomocy mojego specjalnego fontu dla systemu stenografii.
Założeniem systemu stenografii jest to, żeby nie trzeba było odrywać pióra przy pisaniu wyrazu.

Idea systemu:
1. Pisze się wokół linii środkowej ciągnąc kreskę do linii górnej lub dolnej i prawie zawsze (oprócz końca wyrazu) wracając do linii środkowej
1. Kreski mają różne (proste) kształy (w sumie 11)
1. Spółgłoski oznaczane są kreską od linii środkowej w górę lub w dół (różne spółgłoski dla lustrzanej kreski idącej w górę i w dół)
1. Samogłoski oznaczane są kreską od linii górnej lub dolnej do środkowej (ta sama samogłoska może być napisana wracając z góry jak i z dołu po przez lustrzane odbicie)
1. Jeśli występują po sobie dwie spółgłoski lub dwie samogłoski, to między nimi następuje powrót przy pomocy linii pionowej
1. Na końcu spółgłoski może dodany być jeden z dwóch akcentów (kreska i kółko/pętelka)
1. Na końcu samogłoski może dodany być akcent (mała skośna linia i powrót) który zmienia `e` na `ę`, `a` na `ą`
1. Generalnie to można pisać też samogłoski poziomo, na linii środkowej, co jest przydatne gdy samogłoska następuje po samogłosce. Jest to obsłużone przez specjalne ligatury. Póki co tylko `ie` i `ia`.

## Transkrypcja

Pierwszą fazą jest podzielenie tekstu na głoski.

In [1]:
import re

space_pattern = re.compile(r'\s+')
def cleanup(t):
    return re.sub(space_pattern, ' ', t.lower())

cleanup("Nie! 117  dąsać się-pROszę.!")

'nie! 117 dąsać się-proszę.!'

In [2]:
split_pattern = re.compile(r'(sz|cz|rz|dz|dź|dż|ch)')
def dziel(t):
    pos = 0
    for m in re.finditer(split_pattern, t):
        s = m.span()
        if m.start() != pos:
            for c in t[pos:m.start()]:
                yield c
        yield m[1]
        pos = m.end()
    if pos < len(t):
        for c in t[pos:]:
            yield c

print(list(dziel('zaszczute rzeźnickie dżdżownice dźwięczą dzisiaj dzdz ')))

['z', 'a', 'sz', 'cz', 'u', 't', 'e', ' ', 'rz', 'e', 'ź', 'n', 'i', 'c', 'k', 'i', 'e', ' ', 'dż', 'dż', 'o', 'w', 'n', 'i', 'c', 'e', ' ', 'dź', 'w', 'i', 'ę', 'cz', 'ą', ' ', 'dz', 'i', 's', 'i', 'a', 'j', ' ', 'dz', 'dz', ' ']


In [3]:
def czy_samlgloska(s):
    return s in ['a','i','e','o','y','u','ę','ą','ó']

def czy_spolgloska(s):
    return str.isalpha(s) and not czy_samlgloska(s)

Spółgłoski dzielą się na górne i dolne.

In [4]:
gorne = {
    'm',
    's','ś','sz',
    't',
    'l',
    'p',
    'k',
    'w',
    'r',
    'c','ć','cz',
    'dz','dż','dź',
    'x'
}

dolne = {
    'n','ń'
    'z','ż','ź'
    'd',
    'ł',
    'b',
    'g',
    'f',
    'j',
    'h','ch',
    'v',
    'q'
}

Po górnej spółgłosce może występować tylko górna samogłoska (oznaczona upper-case), a po dolnej - dolna.

Jeśli po spółgłosce jest spółgłoska, to między nimi musi znaleźć się linia wracająca do środkowej linii (górna `\` lub dolna `/`).

Tymczasowo podobnie dla samogłosek, póki nie są obsługiwane poziome formy samogłosek.

In [5]:
def klasyfikuj(c):
    sam = czy_samlgloska(c[0])
    sp = czy_spolgloska(c[0])
    inna = not sam and not sp
    gorna = (sp and c in gorne) or inna
    return sam, sp, inna, gorna

def ligatura(prev, c):
    return (prev, c) in [('i','e'), ('i','a'), ('i','ę'), ('i','ą')]

def polacz(iter):
    prev = next(iter)
    prev_sam, prev_sp, prev_inna, prev_gorna = klasyfikuj(prev)
    next_up = prev_sam
    for c in iter:
        sam, sp, inna, gorna = klasyfikuj(c)
        go_up = next_up
        next_up = False
        if sam and prev_sam:
            if ligatura(prev, c):
                next_up = go_up # ten sam case
            else:
                # idziemy w górę
                prev = prev + '↓'
                next_up = True
        elif sam and prev_gorna:
            next_up = True
        elif sp and prev_sp:
            if prev_gorna:
                prev = prev + '↓'
            else:
                prev = prev + '↑'
        # emit before going to next
        if go_up:
            yield prev.upper()
        else:
            yield prev
        prev = c
        prev_sam, prev_sp, prev_inna, prev_gorna = sam, sp, inna, gorna
    # emit last one
    if next_up:
        yield prev.upper()
    else:
        yield prev
        
print(list(polacz(['p','i', 'ę', 'k'].__iter__())))

['p', 'I', 'Ę', 'k']


In [6]:
def konwertuj(s):
    return polacz(dziel(cleanup(s)))

''.join(konwertuj('i tak i dzisiejszy dzień zaczął się pięknie'))

'I tAk I dzIsIEj↑szY dzIEń zaczĄł sIĘ pIĘk↓nie'

In [7]:
with open('8119-0.txt') as f:
    c = 1
    for line in f:
        print(''.join(konwertuj(line)))
        if c > 100:
            break
        c += 1

sIEr↓pIEń nawIEdzEnie p↓tAkI mAnekIny t↓rAk↓tAt O mAnekInach Al↓bo w↓tÓrA 
k↓sIĘga rOdzAju t↓rAk↓tAt O mAnekInach cIĄg dal↓szY t↓rAk↓tAt O mAnekInach 
dokOń↑czEnie nem↓rOd pAn pAn kArOl s↓k↓lEpY cYnamOnowE UlIcA k↓rOkOdylI 
kArAkOny wIchurA noc wIEl↓kIEgo sEzonu 
 
 
 
 
 sIEr↓pIEń 
 
 1 
 
 w lIp↓cU Oj↑cIEc mÓj wYjeż↑dżAł do wÓd I zos↓tAwIAł m↓nie z mAt↓kĄ I 
s↓tAr↓szYm b↑rAtEm na pAs↓t↓wĘ białych Od żarU I OszOłamIAjącYch d↑ni lEt↓nich. 
wEr↓tOwAlIś↓mY, Odurzeni ś↓wIAt↓łem, w tEj wIEl↓kIEj k↓sIĘdzE wAkAc↓ji, k↓tÓrEj 
w↓szYs↓t↓kIE kAr↓tY pAłały Od b↑lAs↓kU I mIAły na d↑nie s↓łod↑kI do Om↓d↑lEnia 
mIĄż↑sz z↑łotYch g↑rUszEk. AdelA w↓rAcAła w ś↓wIEt↓lIs↓tE pOrAn↑kI, jak pOmOna z 
Og↑nia d↑nia rOz↑żag↑wI↓Onego, wYsYpUjąc z kOszYkA bar↓w↓ną UrOdę s↓łoń↑cA - 
l↓ś↓niącE, pEł↑ne wOdy pOd p↓rzej↑rzys↓tĄ s↓kÓr↓kĄ czErEś↓nie, tAjem↓niczE, czAr↓ne 
wIś↓nie, k↓tÓrYch wOń p↓rzek↓rAczAła tO, cO zisz↓czAło sIĘ w s↓mAkU; mOrElE, w 
k↓tÓrYch mIĄż↑szU z↑łotYm był r↓dzEń d↑ługich pOpOłud↑ni; A Obok tEj c

In [9]:
import io

def line_to_char(lines):
    for line in lines:
        for c in dziel(cleanup(line)):
            yield c
        yield '\n'
        

with open('felieton.txt', 'r', encoding='utf-8') as f:
    with open('felieton.out.txt', 'w', encoding='utf-8') as out:
        for c in polacz(line_to_char(f)):
            out.write(c)

## TODO
- poprawić błędy ze sklepów cynamonowych
- statystyki dla par samogłosek i ligatury dla najpopularniejszych

### Pomysły
- strona online do konwersji (2 kolumny)